In [1]:
from qiskit import QuantumCircuit, transpile
# from qiskit.providers.aer import QasmSimulator
# from qiskit.providers.basicaer import QasmSimulatorPy
from qiskit_aer import AerSimulator, Aer
from qiskit.providers.fake_provider import FakeProvider, FakeManila, FakeToronto, FakeJakartaV2
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_provider import IBMProvider
from qiskit.circuit import Parameter
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import QDrift, LieTrotter, SuzukiTrotter
import qiskit.quantum_info as qi
import qiskit
from qiskit.visualization import plot_histogram

import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn.preprocessing as skp
from scipy.signal import savgol_filter
import scipy
import os
import sys
import itertools
from numpy.random import Generator, PCG64
from datetime import date
import pathlib
import inspect
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('..'))
plot_path = f'{module_path}/plots'
sys.path.insert(0, os.path.abspath('../src/'))

from feedforward import QExtremeLearningMachine
from rewinding import QRewindingRC, QRewindingStatevectorRC
from data import DataSource
from noisemodel import thermal_model

# Info about fake backends

In [ ]:
aer_method = 'statevector'
provider = FakeProvider()
backend = provider.get_backend('fake_toronto')
noise_model = NoiseModel.from_backend(backend)
backend = AerSimulator(method = aer_method, noise_model = noise_model)

print(backend.options.noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['sx', 'id', 'reset', 'x', 'measure', 'cx']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2,)), ('id', (3,)), ('id', (4,)), ('id', (5,)), ('id', (6,)), ('id', (7,)), ('id', (8,)), ('id', (9,)), ('id', (10,)), ('id', (11,)), ('id', (12,)), ('id', (13,)), ('id', (14,)), ('id', (15,)), ('id', (16,)), ('id', (17,)), ('id', (18,)), ('id', (19,)), ('id', (20,)), ('id', (21,)), ('id', (22,)), ('id', (23,)), ('id', (24,)), ('id', (25,)), ('id', (26,)), ('sx', (0,)), ('sx', (1,)), ('sx', (2,)), ('sx', (3,)), ('sx', (4,)), ('sx', (5,)), ('sx', (6,)), ('sx', (7,)), ('sx', (8,)), ('sx', (9,)), ('sx', (10,)), ('sx', (11,)), ('sx', (12,)), ('sx', (13,)), ('sx', (14,)), ('sx', (15,)), ('sx', (16,)), ('sx', (17,)), ('sx', (18,)), ('sx', (19,)), ('sx', (20,)), ('sx', (21,)), ('

In [ ]:
noise_model


<NoiseModel on ['sx', 'id', 'reset', 'x', 'measure', 'cx']>

In [ ]:
print(noise_model.noise_instructions)
print(noise_model.basis_gates)
print(noise_model.noise_qubits)


['cx', 'id', 'measure', 'reset', 'sx', 'x']
['cx', 'id', 'reset', 'rz', 'sx', 'x']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


In [ ]:
# https://qiskit.org/documentation/stubs/qiskit.providers.models.PulseBackendConfiguration.html#pulsebackendconfiguration
backend = provider.get_backend('fake_toronto')
print(backend.configuration().coupling_map)
print(backend.configuration().basis_gates)
print(backend.configuration().n_qubits)
print(backend.configuration().gates)

[[0, 1], [1, 0], [1, 2], [1, 4], [2, 1], [2, 3], [3, 2], [3, 5], [4, 1], [4, 7], [5, 3], [5, 8], [6, 7], [7, 4], [7, 6], [7, 10], [8, 5], [8, 9], [8, 11], [9, 8], [10, 7], [10, 12], [11, 8], [11, 14], [12, 10], [12, 13], [12, 15], [13, 12], [13, 14], [14, 11], [14, 13], [14, 16], [15, 12], [15, 18], [16, 14], [16, 19], [17, 18], [18, 15], [18, 17], [18, 21], [19, 16], [19, 20], [19, 22], [20, 19], [21, 18], [21, 23], [22, 19], [22, 25], [23, 21], [23, 24], [24, 23], [24, 25], [25, 22], [25, 24], [25, 26], [26, 25]]
['id', 'rz', 'sx', 'x', 'cx', 'reset']
27
[GateConfig(id, [], gate id q { U(0, 0, 0) q; }, [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26]]), GateConfig(rz, ['theta'], gate rz(theta) q { U(0, 0, theta) q; }, [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26]]), GateConfig(s

In [ ]:
# https://qiskit.org/documentation/locale/de_DE/stubs/qiskit.providers.models.BackendProperties.html#backendproperties
backend = provider.get_backend('fake_toronto')
print(backend.properties())



In [ ]:
print(backend.properties().qubits[0])
print(backend.properties().gates[0].name)
print(backend.properties().gates[0].qubits)
print(backend.properties().gates[0].gate)
print(backend.properties().gates[0].parameters)

[Nduv(datetime.datetime(2021, 3, 15, 2, 45, 30, tzinfo=tzoffset(None, -14400)), T1, us, 56.40156543513177), Nduv(datetime.datetime(2021, 3, 15, 2, 57, 43, tzinfo=tzoffset(None, -14400)), T2, us, 50.69720583030928), Nduv(datetime.datetime(2021, 3, 15, 14, 16, 30, tzinfo=tzoffset(None, -14400)), frequency, GHz, 5.224962212570894), Nduv(datetime.datetime(2021, 3, 15, 14, 16, 30, tzinfo=tzoffset(None, -14400)), anharmonicity, GHz, -0.33434735824128314), Nduv(datetime.datetime(2021, 3, 15, 2, 31, 46, tzinfo=tzoffset(None, -14400)), readout_error, , 0.057499999999999996), Nduv(datetime.datetime(2021, 3, 15, 2, 31, 46, tzinfo=tzoffset(None, -14400)), prob_meas0_prep1, , 0.0504), Nduv(datetime.datetime(2021, 3, 15, 2, 31, 46, tzinfo=tzoffset(None, -14400)), prob_meas1_prep0, , 0.06459999999999999), Nduv(datetime.datetime(2021, 3, 15, 2, 31, 46, tzinfo=tzoffset(None, -14400)), readout_length, ns, 85617.77777777777)]
id0
[0]
id
[Nduv(datetime.datetime(2021, 3, 15, 3, 22, 2, tzinfo=tzoffset(None,

In [ ]:
print(backend.properties().gate_error(gate='x', qubits=[0]))
print(backend.properties().gate_error(gate='cx', qubits=[0,1]))
print(backend.properties().gate_length(gate='cx', qubits=[0,1]))

0.00024166799076583536
0.008945423825359594
3.868444444444444e-06


# Building Noisemodel from scratch

In [ ]:
# # Create an empty noise model
# noise_model = NoiseModel()

# # Add depolarizing error to all single qubit u1, u2, u3 gates
# error = depolarizing_error(0.05, 1)
# noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])


# # Measurement miss-assignement probabilities
# p0given1 = 0.1
# p1given0 = 0.05

# ReadoutError([[1 - p1given0, p1given0], [p0given1, 1 - p0given1]])

In [ ]:
noise_thermal = thermal_model(nqubits=5)

sim_thermal = AerSimulator(noise_model=noise_thermal)